In [ ]:
import os
import cv2 as cv
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from keras import Sequential
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [ ]:
root = "drive/MyDrive/tgm/"

vuongs = os.listdir(root + "vuongs/")
trons = os.listdir(root + "trons/")
tamgiacs = os.listdir(root + "tamgiacs/")

vuongs_test = os.listdir(root + "test/vuongs/")
trons_test = os.listdir(root + "test/trons/")
tamgiacs_test = os.listdir(root + "test/tamgiacs/")

In [ ]:
vuongs_array = []
trons_array = []
tamgiacs_array = []

kernel = np.ones((7, 7), np.uint8)

for vuong in vuongs:
    img = np.asarray(Image.open(root + "vuongs/" + vuong))
    gray = .2989 * img[:, :, 0] + .5870 * img[:, :, 1] + .1140 * img[:, :, 2]
    gray = 255 - gray
    dilation1 = cv.dilate(gray, kernel, iterations=1)
    dilation2 = cv.dilate(gray, kernel, iterations=2)
    vuongs_array.extend([gray, dilation1, dilation2])

for tron in trons:
    img = np.asarray(Image.open(root + "trons/" + tron))
    gray = .2989 * img[:, :, 0] + .5870 * img[:, :, 1] + .1140 * img[:, :, 2]
    gray = 255 - gray
    dilation1 = cv.dilate(gray, kernel, iterations=1)
    dilation2 = cv.dilate(gray, kernel, iterations=2)
    trons_array.extend([gray, dilation1, dilation2])

for tamgiac in tamgiacs:
    img = np.asarray(Image.open(root + "tamgiacs/" + tamgiac))
    gray = .2989 * img[:, :, 0] + .5870 * img[:, :, 1] + .1140 * img[:, :, 2]
    gray = 255 - gray
    dilation1 = cv.dilate(gray, kernel, iterations=1)
    dilation2 = cv.dilate(gray, kernel, iterations=2)
    tamgiacs_array.extend([gray, dilation1, dilation2])

vuongs_array = np.array(vuongs_array)
trons_array = np.array(trons_array)
tamgiacs_array = np.array(tamgiacs_array)

xtrain = np.concatenate([vuongs_array, trons_array, tamgiacs_array], axis=0)
xtrain = np.expand_dims(xtrain, axis=-1)
print(xtrain.shape)

vuongs_label = np.full(vuongs_array.shape[:1], 0.)
trons_label = np.full(trons_array.shape[:1], 1.)
tamgiacs_label = np.full(tamgiacs_array.shape[:1], 2.)

ytrain = np.concatenate([vuongs_label, trons_label, tamgiacs_label], axis=0)
ytrain.shape

(1080, 300, 300, 1)


(1080,)

In [ ]:
vuongs_array = []
trons_array = []
tamgiacs_array = []

kernel = np.ones((7, 7), np.uint8)

for vuong in vuongs_test:
    img = np.asarray(Image.open(root + "test/vuongs/" + vuong))
    gray = .2989 * img[:, :, 0] + .5870 * img[:, :, 1] + .1140 * img[:, :, 2]
    gray = 255 - gray
    dilation1 = cv.dilate(gray, kernel, iterations=1)
    dilation2 = cv.dilate(gray, kernel, iterations=2)
    vuongs_array.extend([gray, dilation1, dilation2])

for tron in trons_test:
    img = np.asarray(Image.open(root + "test/trons/" + tron))
    gray = .2989 * img[:, :, 0] + .5870 * img[:, :, 1] + .1140 * img[:, :, 2]
    gray = 255 - gray
    dilation1 = cv.dilate(gray, kernel, iterations=1)
    dilation2 = cv.dilate(gray, kernel, iterations=2)
    trons_array.extend([gray, dilation1, dilation2])

for tamgiac in tamgiacs_test:
    img = np.asarray(Image.open(root + "test/tamgiacs/" + tamgiac))
    gray = .2989 * img[:, :, 0] + .5870 * img[:, :, 1] + .1140 * img[:, :, 2]
    gray = 255 - gray
    dilation1 = cv.dilate(gray, kernel, iterations=1)
    dilation2 = cv.dilate(gray, kernel, iterations=2)
    tamgiacs_array.extend([gray, dilation1, dilation2])

vuongs_array = np.array(vuongs_array)
trons_array = np.array(trons_array)
tamgiacs_array = np.array(tamgiacs_array)

xtest = np.concatenate([vuongs_array, trons_array, tamgiacs_array], axis=0)
xtest = np.expand_dims(xtest, axis=-1)
print(xtest.shape)

vuongs_label = np.full(vuongs_array.shape[:1], 0.)
trons_label = np.full(trons_array.shape[:1], 1.)
tamgiacs_label = np.full(tamgiacs_array.shape[:1], 2.)

ytest = np.concatenate([vuongs_label, trons_label, tamgiacs_label], axis=0)
print(ytest.shape)

(180, 300, 300, 1)
(180,)


In [ ]:
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=.2, random_state=42)

print(xtrain.shape, xval.shape)

y_train = to_categorical(ytrain)
y_val = to_categorical(yval)
y_test = to_categorical(ytest)

train_generator = ImageDataGenerator(rotation_range=5, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=5, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=5, horizontal_flip=True)

train_generator.fit(xtrain)
val_generator.fit(xval)
test_generator.fit(xtest)

input_shape = xtrain[0].shape

(864, 300, 300, 1) (216, 300, 300, 1)


In [ ]:
model = Sequential()
model.add(Conv2D(16, (5, 5), input_shape=input_shape))
model.add(Activation("relu"))
model.add(Dropout(0.3))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (5, 5)))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (5, 5)))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(1028))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation("softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 296, 296, 16)      416       
_________________________________________________________________
activation (Activation)      (None, 296, 296, 16)      0         
_________________________________________________________________
dropout (Dropout)            (None, 296, 296, 16)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 148, 148, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 144, 144, 16)      6416      
_________________________________________________________________
activation_1 (Activation)    (None, 144, 144, 16)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 144, 144, 16)      6

In [ ]:
batch_size = 64
epochs = 10

model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(train_generator.flow(xtrain, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(xval, y_val, batch_size=batch_size), verbose=1)


print(history.history)
print(model.evaluate(xtest, y_test))

Epoch 1/10
14/14 [==============================] - 34s 733ms/step - loss: 0.9710 - accuracy: 0.6477 - val_loss: 5.1518 - val_accuracy: 0.8102
Epoch 2/10
14/14 [==============================] - 8s 560ms/step - loss: 0.2694 - accuracy: 0.8894 - val_loss: 2.3916 - val_accuracy: 0.7963
Epoch 3/10
14/14 [==============================] - 8s 559ms/step - loss: 0.2209 - accuracy: 0.9250 - val_loss: 0.7970 - val_accuracy: 0.8519
Epoch 4/10
14/14 [==============================] - 8s 558ms/step - loss: 0.1156 - accuracy: 0.9570 - val_loss: 0.2067 - val_accuracy: 0.9259
Epoch 5/10
14/14 [==============================] - 8s 553ms/step - loss: 0.1022 - accuracy: 0.9606 - val_loss: 0.1754 - val_accuracy: 0.9352
Epoch 6/10
14/14 [==============================] - 8s 557ms/step - loss: 0.0720 - accuracy: 0.9709 - val_loss: 0.0327 - val_accuracy: 0.9861
Epoch 7/10
14/14 [==============================] - 8s 560ms/step - loss: 0.0648 - accuracy: 0.9748 - val_loss: 0.0174 - val_accuracy: 0.9907
Epoch

In [ ]:
model.save("model_tgm.h5")